# Описание задачи

В некоторой архитектуре нейронной сети зависимость выхода $y \in \mathbb{R}$ от входа $(x_1, x_2) \in \mathbb{R}^2$ можно записать следующим образом:
$$
\begin{align*}
&f_1(x_1, x_2) = x_1 + x_2,\\
&f_2(x_1, x_2) = x_1 \cdot x_2,\\
&g_1(x_1, x_2) = \tan( f_1(x_1, x_2) + f_2(x_1, x_2) + 100),\\
&g_2(x_1, x_2) = f_1(x_1, x_2) \cdot f_2(x_1, x_2),\\
&y(x_1, x_2) = p(g_1(x_1,x_2), g_2(x_1, x_2)),
\end{align*}
$$
где $p(g_1, g_2)-$ некоторая неизвестная функция.

С помощью [механизма автоматического дифференцирования в PyTorch](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#) необходимо вычислить значения частных производных в разных точках:
$$
\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, 
$$

Для этого в файле `dev.json` приведены значения $\frac{\partial p}{\partial g_1}, \frac{\partial p}{\partial g_2}$ (обозначены как `dpdg1, dpdg2`), вычисленные при известных значениях входа. 

Нужно:

1. Загрузить данные из файла `dev.json`
2. Посчитать значения нужных производных для каждой точки.
3. Сохранить результат в формате CSV. Файл должен иметь следующую структуру:
```
id,dx1,dx2
....
```
`id` - это значение `id` из `dev.json`, `dx1, dx2`- значение производных, вычисленные в точке `id` из `dev.json.`

Ответ проверяется с помощью функции [math.isclose](https://docs.python.org/3/library/math.html#math.isclose) с точностью до `1e-8`. 

In [1]:
import torch
import json

In [2]:
with open("./dev.json") as f:
    dev = json.load(f)

In [3]:
# Считываем x1 и x2
x1 = torch.tensor([d['x1'] for d in dev], requires_grad=True)
x2 = torch.tensor([d['x2'] for d in dev], requires_grad=True)

# Считываем dpdg1 и dpdg2
grad_g1 = torch.tensor([d['dpdg1'] for d in dev])
grad_g2 = torch.tensor([d['dpdg2'] for d in dev])

# Определяем значения функций в точках x1 и x2
f1 = x1 + x2
f2 = x1 * x2
g1 = torch.tan(f1 + f2 + 100)
g2 = f1 * f2

Формулы:   
$$
\frac{\partial y}{\partial x_1} = \frac{\partial p}{\partial g_1}*\frac{\partial g_1}{\partial x_1} + \frac{\partial p}{\partial g_2}*\frac{\partial g_2}{\partial x_1} (1)
$$  

$$
\frac{\partial y}{\partial x_2} = \frac{\partial p}{\partial g_1}*\frac{\partial g_1}{\partial x_2} + \frac{\partial p}{\partial g_2}*\frac{\partial g_2}{\partial x_2} (2)
$$  


In [4]:
# Получаем градиенты по g1
g1.sum().backward(retain_graph=True)
x1_grad = x1.grad
x2_grad = x2.grad

# Считаем первые слагаемые в формуле (1) и (2) соответственно
a1 = grad_g1 * x1_grad
a2 = grad_g1 * x2_grad
x1.grad.zero_()
x2.grad.zero_()

# Получаем градиенты по g2
g2.sum().backward(retain_graph=True)
x1_grad = x1.grad
x2_grad = x2.grad

# Считаем вторые слагаемые в формуле (1) и (2) соответственно
b1 = grad_g2 * x1_grad
b2 = grad_g2 * x2_grad

In [5]:
dydx1 = a1 + b1
dydx2 = a2 + b2
print(f"dy/dx1 = {dydx1.tolist()}")
print(f"dy/dx2 = {dydx2.tolist()}")

dy/dx1 = [1.560255765914917, 4.967438220977783, 1397.6683349609375, 59.70181655883789, 91.28831481933594, -0.7811862230300903, -0.35697290301322937, 0.45325127243995667, 5.779941082000732, 0.7605778574943542]
dy/dx2 = [2.0092973709106445, 3.0528149604797363, 1626.5355224609375, 46.438087463378906, 85.2686767578125, -0.6984623074531555, -0.3496031165122986, 0.2956584393978119, 5.595616340637207, 0.577911376953125]
